In [1]:
import os
from pathlib import Path
os.chdir('../')
%pwd

'c:\\Users\\Saumil\\Git\\text_summarizer'

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path
    

In [8]:

from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_dir

In [9]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_dir([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_dir([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_url = config.source_url,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir)
        
        return data_ingestion_config



In [10]:
import os
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size



In [11]:
class DataIngestion:
    def __init__(
        self,
        config: DataIngestionConfig):
        self.config = config
    
    def download_data(self):
        if not os.path.exists(self.config.local_data_file):
            file, headers = request.urlretrieve(
                url = self.config.source_url, 
                filename = self.config.local_data_file)
            logger.info(f'Downloaded {file} to {self.config.local_data_file} using {headers}')
            logger.info(f'Size of {self.config.local_data_file} is {get_size(Path(self.config.local_data_file))}')
        else:
            logger.info(f'{self.config.local_data_file} already exists of size {get_size(self.config.local_data_file)}')
    
    def unzip_data(self):
        unzip_path = Path(self.config.unzip_dir)
        if not os.path.exists(unzip_path):
            create_dir([unzip_path])
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
            logger.info(f'Unzipped {self.config.local_data_file} to {unzip_path}')
    

In [12]:
try:
    config = ConfigurationManager()
    data_ingestion = DataIngestion(config.get_data_ingestion_config())
    data_ingestion.download_data()
    data_ingestion.unzip_data()
except Exception as e:
    raise e

[2024-01-19 18:22:36,541: INFO: common: Read config\config.yaml successfully]
[2024-01-19 18:22:36,543: INFO: common: Read params.yaml successfully]
[2024-01-19 18:22:36,544: INFO: common: artifacts already exists]
[2024-01-19 18:22:36,545: INFO: common: artifacts/data_ingestion created]
[2024-01-19 18:22:38,019: INFO: 2716565283: Downloaded artifacts/data_ingestion/data.zip to artifacts/data_ingestion/data.zip using Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 77BA:14F093:31DEB06:33E45C5:65AABDEA
Accept-Ranges: bytes
Date: Fri, 19 Jan 2024 18:22:37 GMT
Via: 1.1 varnish
X-Served-By: cache-lon4268-LON
X-Cache: MISS
X-Cache-Hits: 0
X-Ti